You can run this notebook directly on Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DaniAffCH/Vessel-Geometric-Transformers/blob/main/main.ipynb)

In [1]:
import sys

COLAB_RUNTIME = 'google.colab' in sys.modules
if COLAB_RUNTIME:
    !git clone https://github.com/DaniAffCH/Vessel-Geometric-Transformers.git
    !mv Vessel-Geometric-Transformers/* . 
    !pip install -q -r requirements.txt
else:
    !pip install -q -r requirements.txt
    !pre-commit autoupdate
    !pre-commit install



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
[https://github.com/psf/black] already up to date!
[https://github.com/pycqa/isort] already up to date!
[https://github.com/PyCQA/flake8] already up to date!
[https://github.com/pre-commit/mirrors-mypy] already up to date!
pre-commit installed at .git/hooks/pre-commit


Loading the configuration

In [2]:
from src.utils import load_config
import os
from config import DatasetConfig, TrainerConfig, BaselineConfig

config_path = os.path.join("config","config.yaml")

config = load_config(config_path)
dataset_config: DatasetConfig = config.dataset
trainer_config: TrainerConfig = config.trainer
baseline_config: BaselineConfig = config.baseline

Loading the dataset

In [3]:
from src.data import VesselDataModule


data = VesselDataModule(dataset_config)
print(f'Train size: {len(data.train_set)}')
print(f'Validation size: {len(data.val_set)}')
print(f'Test size: {len(data.test_set)}')
print(data.train_set[2])

Train size: 2999
Validation size: 599
Test size: 401
Data(pos=[8397, 3], wss=[8397, 3], pressure=[8397], face=[3, 16790], inlet_index=[214], label=Category.Single)


# TODO: analisi dati preliminare. Classi bilanciate? Statistiche sul dataset? Qualche plot figo.

In [9]:
from src.lib.geometricAlgebraOperations import *
from src.lib import PointGeometricAlgebra, PlaneGeometricAlgebra

# Example vectors from your dataset
c1 = PointGeometricAlgebra.fromElement(data.train_set[0].pos[0].unsqueeze(0))[0]
c2 = PointGeometricAlgebra.fromElement(data.train_set[1].pos[0].unsqueeze(0))[0]


#GeometricProduct.apply(c1,c2)
#Dual.apply(c1)


(tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0150, -0.1882,  0.0018,  1.0000,  0.0000]),
 tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0150, -0.1882,  0.0018,  1.0000,  0.0000]))

Testing

In [4]:
from src.lib import PointGeometricAlgebra, TranslationGeometricAlgebra, ScalarGeometricAlgebra, PlaneGeometricAlgebra

print(data.train_set[0].pos[0])
print(PointGeometricAlgebra.fromElement(data.train_set[0].pos[0].unsqueeze(0)))
print()

print(TranslationGeometricAlgebra.fromElement(data.train_set[1].wss)[:, :8])
print()

print(ScalarGeometricAlgebra.fromElement(data.train_set[0].pressure))
print()

print(PlaneGeometricAlgebra.fromElement(data.train_set[0].face.T))

tensor([ 0.0150, -0.1882,  0.0018])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0150, -0.1882,  0.0018,  1.0000,  0.0000]])

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

tensor([[133520.2500,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        [133502.0156,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        [133503.4844,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        ...,
        [133278.6875,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        [133279.0469,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        [133595.

Training Loop

In [5]:
from src.trainer import VesselTrainer
from src.models import BaselineTransformer

model = BaselineTransformer(baseline_config)
trainer = VesselTrainer(trainer_config)

trainer.fit(model, data)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: danieleaffinita2000. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | encoder        | TransformerEncoder | 2.2 K  | train
1 | embedder       | Linear             | 272    | tr

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Epoch 5: 100%|██████████| 750/750 [00:19<00:00, 39.30it/s, v_num=osgj, val/loss=8.91e-5, val/acc=1.000, val/f1=1.000, train/loss=7.87e-5, train/acc=1.000, train/f1=1.000]  

Epoch 5, global step 4500: 'val/loss' reached 0.00009 (best 0.00009), saving model to '/home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=5-step=4500.ckpt' as top 1


Epoch 6: 100%|██████████| 750/750 [00:19<00:00, 39.45it/s, v_num=osgj, val/loss=6.08e-5, val/acc=1.000, val/f1=1.000, train/loss=3.8e-5, train/acc=1.000, train/f1=1.000] 

Monitored metric val/loss did not improve in the last 3 records. Best score: 0.001. Signaling Trainer to stop.
Epoch 6, global step 5250: 'val/loss' reached 0.00006 (best 0.00006), saving model to '/home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt/epoch=6-step=5250.ckpt' as top 1


Epoch 6: 100%|██████████| 750/750 [00:19<00:00, 39.42it/s, v_num=osgj, val/loss=6.08e-5, val/acc=1.000, val/f1=1.000, train/loss=3.8e-5, train/acc=1.000, train/f1=1.000]


In [7]:
trainer.test(model, data)

Restoring states from the checkpoint path at ckpt/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ckpt/last.ckpt


Testing DataLoader 0: 100%|██████████| 101/101 [00:01<00:00, 65.71it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │            1.0            │
│          test/f1          │            1.0            │
│         test/loss         │  0.00021707075939048082   │
└───────────────────────────┴───────────────────────────┘